# Data Acquisition and Cleaning
This document presents the work done in order to obtain a usable dataset. Indeed, the Million Song Dataset (MSD) was lackluster regarding song features (danceability, energy, etc), which were all set to 0. That's why we decided to take additional steps to improve this dataset.    
In the first part, we import data from the MSD and we decide to keep only some values of interest from this dataset. Next, we query the Spotify API in order to obtain additional information about each song. In the meantime, we also add genre classification for each song from 3 datasets built around the MSD. Finally, we generate a csv which will be our main resource of data for the rest of the project.

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import numpy as np

%load_ext autoreload
%autoreload 2

The following function aims at creating a connection to SQLite database.

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return None

We have a quick look at the columns we have at our disposal.

In [3]:
database = "data/track_metadata.db"
conn = create_connection(database)
cur = conn.cursor()
cur.execute("PRAGMA table_info(songs)")
rows = cur.fetchall()
print(rows)

[(0, 'track_id', 'text', 0, None, 1), (1, 'title', 'text', 0, None, 0), (2, 'song_id', 'text', 0, None, 0), (3, 'release', 'text', 0, None, 0), (4, 'artist_id', 'text', 0, None, 0), (5, 'artist_mbid', 'text', 0, None, 0), (6, 'artist_name', 'text', 0, None, 0), (7, 'duration', 'real', 0, None, 0), (8, 'artist_familiarity', 'real', 0, None, 0), (9, 'artist_hotttnesss', 'real', 0, None, 0), (10, 'year', 'int', 0, None, 0)]


Request the dataset and put the data in a dataframe.

In [4]:
cur.execute("SELECT track_id, song_id, artist_id, duration, artist_hotttnesss, year FROM songs ORDER BY track_id")
rows = cur.fetchall()
songs = pd.DataFrame(rows, columns=['track_id', 'song_id', 'artist_id', 'duration', 'artist_hotttnesss', 'year'])

The 3 following cells merge our data from genres classification coming from 3 datasets.

In [5]:
track_genre_cd1 = pd.read_csv('data/msd_tagtraum_cd1.cls', sep='\t', names=['track_id', 'genre1_cd1', 'genre2_cd1'])
songs = track_genre_cd1.merge(songs, on='track_id', how='right')

In [6]:
track_genre_cd1 = pd.read_csv('data/msd_tagtraum_cd2.cls', sep='\t', names=['track_id', 'genre1_cd2', 'genre2_cd2'])
songs = track_genre_cd1.merge(songs, on='track_id', how='right')

In [7]:
track_genre_cd1 = pd.read_csv('data/msd_tagtraum_cd2c.cls', sep='\t', names=['track_id', 'genre1_cd2c', 'genre2_cd2c'])
songs = track_genre_cd1.merge(songs, on='track_id', how='right')

We take look at the analysis of the songs from the MSD

In [8]:
msd_summary_file = pd.HDFStore("data/msd_summary_file.h5")
songs_analysis = msd_summary_file.get('/analysis/songs')
songs_analysis.columns

Index(['analysis_sample_rate', 'audio_md5', 'danceability', 'duration',
       'end_of_fade_in', 'energy', 'idx_bars_confidence', 'idx_bars_start',
       'idx_beats_confidence', 'idx_beats_start', 'idx_sections_confidence',
       'idx_sections_start', 'idx_segments_confidence',
       'idx_segments_loudness_max', 'idx_segments_loudness_max_time',
       'idx_segments_loudness_start', 'idx_segments_pitches',
       'idx_segments_start', 'idx_segments_timbre', 'idx_tatums_confidence',
       'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode',
       'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature',
       'time_signature_confidence', 'track_id'],
      dtype='object')

And add the data we find interesting. Indeed a lot of columns in this dataset are empty and cannot be used.

In [9]:
songs_analysis = songs_analysis[['track_id', 'loudness', 'mode', 'tempo', 'key']]
songs = songs_analysis.merge(songs, on='track_id')
del(songs_analysis)

We do the same operation for the metadata.

In [10]:
songs_metadata = msd_summary_file.get('/metadata/songs')
songs_metadata.columns

Index(['analyzer_version', 'artist_7digitalid', 'artist_familiarity',
       'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location',
       'artist_longitude', 'artist_mbid', 'artist_name', 'artist_playmeid',
       'genre', 'idx_artist_terms', 'idx_similar_artists', 'release',
       'release_7digitalid', 'song_hotttnesss', 'song_id', 'title',
       'track_7digitalid'],
      dtype='object')

In [11]:
songs_metadata = songs_metadata[['song_hotttnesss', 'song_id', 'artist_latitude', 'artist_location', 'artist_longitude']]
songs = songs_metadata.merge(songs, on='song_id')
del(songs_metadata)

And we take a look at the data we have gathered until now.

In [12]:
songs.head()

,song_hotttnesss,song_id,artist_latitude,artist_location,artist_longitude,track_id,loudness,mode,tempo,key,genre1_cd2c,genre2_cd2c,genre1_cd2,genre2_cd2,genre1_cd1,genre2_cd1,artist_id,duration,artist_hotttnesss,year
0,0.542899,SOQMMHC12AB0180CB8,NaN,,NaN,TRMMMYQ128F932D901,-4.829,0,87.002,10,NaN,NaN,NaN,NaN,NaN,NaN,ARYZTJS1187B98C555,252.05506,0.394032,2003
1,0.299877,SOVFVAK12A8C1350D9,NaN,,NaN,TRMMMKD128F425225D,-10.555,1,150.778,9,NaN,NaN,NaN,NaN,NaN,NaN,ARMVN3U1187FB3A1EB,156.55138,0.356992,1995
2,0.617871,SOGTUKN12AB017F4F1,55.8578,"Glasgow, Scotland",-4.24251,TRMMMRX128F93187D9,-2.060,1,177.768,7,NaN,NaN,Electronic,Rap,Electronic,Rap,ARGEKB01187FB50750,138.97098,0.437504,2006
3,NaN,SOBNYVR12A8C13558C,NaN,,NaN,TRMMMCH128F425532C,-4.654,1,87.433,7,NaN,NaN,NaN,NaN,NaN,NaN,ARNWYLR1187B9B2F9C,145.05751,0.372349,2003
4,NaN,SOHSBXH12A8C13B0DF,NaN,,NaN,TRMMMWA128F426B589,-7.806,0,140.035,5,NaN,NaN,NaN,NaN,NaN,NaN,AREQDTE1269FB37231,514.29832,0.000000,0


In order to gather data from the Spotify API we have some scripts in auxiliary files (stored in the folder spotify_requests_tools). Using these tools, we created two csv 'feature_songs.csv' and 'track_year_popularity.csv' which contain additional information for each song.

In [13]:
spotify_data = pd.read_csv('data/feature_songs.csv')
spotify_data.columns

Index(['song_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms'],
      dtype='object')

In [14]:
# Replace the unknown values(zeros) by NaN
for column in spotify_data.columns:
    spotify_data.loc[spotify_data[column] == 0, column] = np.nan

In [15]:
songs2 = songs.merge(spotify_data, how='left', on='song_id')

Again taking a look at the data we have until now

In [16]:
songs2.head()

,song_hotttnesss,song_id,artist_latitude,artist_location,artist_longitude,track_id,loudness_x,mode_x,tempo_x,key_x,...,key_y,loudness_y,mode_y,speechiness,acousticness,instrumentalness,liveness,valence,tempo_y,duration_ms
0,0.542899,SOQMMHC12AB0180CB8,NaN,,NaN,TRMMMYQ128F932D901,-4.829,0,87.002,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.299877,SOVFVAK12A8C1350D9,NaN,,NaN,TRMMMKD128F425225D,-10.555,1,150.778,9,...,9.0,-10.303,1.0,0.0462,0.002230,0.090,0.0869,0.880,149.971,157933.0
2,0.617871,SOGTUKN12AB017F4F1,55.8578,"Glasgow, Scotland",-4.24251,TRMMMRX128F93187D9,-2.060,1,177.768,7,...,4.0,-1.711,NaN,0.0662,0.000045,0.952,0.3430,0.621,177.745,138960.0
3,NaN,SOBNYVR12A8C13558C,NaN,,NaN,TRMMMCH128F425532C,-4.654,1,87.433,7,...,7.0,-4.302,1.0,0.1470,0.130000,NaN,0.1150,0.962,174.013,145032.0
4,NaN,SOHSBXH12A8C13B0DF,NaN,,NaN,TRMMMWA128F426B589,-7.806,0,140.035,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
songs2.iloc[1, :]

song_hotttnesss                0.299877
song_id              SOVFVAK12A8C1350D9
artist_latitude                     NaN
artist_location                        
artist_longitude                    NaN
track_id             TRMMMKD128F425225D
loudness_x                      -10.555
mode_x                                1
tempo_x                         150.778
key_x                                 9
genre1_cd2c                         NaN
genre2_cd2c                         NaN
genre1_cd2                          NaN
genre2_cd2                          NaN
genre1_cd1                          NaN
genre2_cd1                          NaN
artist_id            ARMVN3U1187FB3A1EB
duration                        156.551
artist_hotttnesss              0.356992
year                               1995
danceability                      0.489
energy                            0.583
key_y                                 9
loudness_y                      -10.303
mode_y                                1


In [18]:
spotify_year_pop = pd.read_csv('data/track_year_popularity.csv')
final_merge = songs2.merge(spotify_year_pop.drop_duplicates(['song_id'], keep='last'), how='left', on='song_id')

Finally we take a look at our data and save them into a csv file.

In [19]:
final_merge.columns

Index(['song_hotttnesss', 'song_id', 'artist_latitude', 'artist_location',
       'artist_longitude', 'track_id', 'loudness_x', 'mode_x', 'tempo_x',
       'key_x', 'genre1_cd2c', 'genre2_cd2c', 'genre1_cd2', 'genre2_cd2',
       'genre1_cd1', 'genre2_cd1', 'artist_id', 'duration',
       'artist_hotttnesss', 'year', 'danceability', 'energy', 'key_y',
       'loudness_y', 'mode_y', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo_y', 'duration_ms',
       'track_popularity', 'album_release'],
      dtype='object')

In [20]:
final_merge.to_csv('final_merge.csv')

In [21]:
final_merge.shape

(1007807, 34)

It seems that we have gained some rows while joining the datasets. This may be due to duplicate IDs.

In [22]:
final_merge.loc[2,:]

song_hotttnesss                0.617871
song_id              SOGTUKN12AB017F4F1
artist_latitude                 55.8578
artist_location       Glasgow, Scotland
artist_longitude               -4.24251
track_id             TRMMMRX128F93187D9
loudness_x                        -2.06
mode_x                                1
tempo_x                         177.768
key_x                                 7
genre1_cd2c                         NaN
genre2_cd2c                         NaN
genre1_cd2                   Electronic
genre2_cd2                          Rap
genre1_cd1                   Electronic
genre2_cd1                          Rap
artist_id            ARGEKB01187FB50750
duration                        138.971
artist_hotttnesss              0.437504
year                               2006
danceability                      0.662
energy                            0.823
key_y                                 4
loudness_y                       -1.711
mode_y                              NaN
